In [3]:
# %matplotlib inline
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2

import skimage
import skimage.io

In [2]:
print tf.__version__

1.4.1


In [4]:
os.environ['CUDA_VISIBLE_DEVICES'] = '2' 
tf.reset_default_graph()
config = tf.ConfigProto(log_device_placement=True)
config.gpu_options.per_process_gpu_memory_fraction = 0.1
s = tf.InteractiveSession(config=config)

In [4]:
s.graph, tf.get_default_graph()

(<tensorflow.python.framework.ops.Graph at 0x7f6f9205f050>,
 <tensorflow.python.framework.ops.Graph at 0x7f6f9205f050>)

In [5]:
# about read image

In [6]:
path = "./juri1.jpg"
juri = tf.image.decode_image(tf.read_file(path), channels=3, name="decode_img")

In [10]:
img = s.run(juri)

In [15]:
img1 = cv2.imread(path)

In [16]:
cv2.imshow("img", img)
cv2.imshow("img1", img1)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [17]:
juri1 = juri - 10

In [20]:
a = [juri, juri]

In [22]:
a[0][:,:,1] = a[0][:,:,1] - 10

TypeError: 'Tensor' object does not support item assignment

In [41]:
# about string
a = tf.constant(dtype=tf.string, value="a,b,c", shape=[])
b = tf.constant(dtype=tf.string, value="d,e,f")

In [42]:
a.shape, b.shape

(TensorShape([]), TensorShape([]))

In [46]:
s.run(tf.string_join([a,b], separator='/')) ,\
s.run(tf.reduce_join([a,b]))

('a,b,c/d,e,f', 'a,b,cd,e,f')

In [44]:
# about variable assign_add
v = tf.Variable(0, trainable=False)
v_add = v.assign_add(1)
v.initializer.run()
for i in range(16):
    print(s.run(v_add))
    print s.run(v)

1
1
2
2
3
3
4
4
5
5
6
6
7
7
8
8
9
9
10
10
11
11
12
12
13
13
14
14
15
15
16
16


In [6]:
# about decay
global_step = tf.Variable(0, trainable=False)
initial_learning_rate = 0.1 #初始学习率
learning_rate = tf.train.exponential_decay(initial_learning_rate,
                                           global_step=global_step,
                                           decay_steps=10,decay_rate=0.9, staircase=False)
# opt = tf.train.GradientDescentOptimizer(learning_rate)

add_global = global_step.assign_add(1)
s.run(tf.global_variables_initializer())
print(s.run(learning_rate))
for i in range(10):
    add_g, rate = s.run([add_global, learning_rate])
    print add_g, rate

0.1
1 0.09895193
2 0.09791484
3 0.09688862
4 0.095873155
5 0.094868325
6 0.09387404
7 0.092890166
8 0.09191661
9 0.09095325
10 0.089999996


In [8]:
0.1 * pow(0.9, 10/10.)

0.09000000000000001

In [ ]:
learning_rate

In [ ]:
# about loss

In [11]:
def myprob(score):
    return 1./(1+np.exp(-score))

def myce(x, z):
    return z * -np.log( myprob(x) ) + (1-z) * -np.log(1-myprob(x))

def myce1(x, z):
    return np.max(x,0) - x*z + np.log(1+np.exp(-np.abs(x)))

In [8]:
prob = tf.constant([2,3,4], dtype=tf.float32)
label = tf.constant([1,2,0], dtype=tf.float32)
ce = tf.nn.sigmoid_cross_entropy_with_logits(logits=prob, labels=label)

In [9]:
ce.eval()

array([ 0.126928 , -2.9514127,  4.01815  ], dtype=float32)

In [16]:
myprob(1)

0.7310585786300049

In [13]:
x = 0
z = 1
myce1(x,z)

0.6931471805599453

In [14]:
x = 0
myce(x, 1), myce1(x, 1)

(0.6931471805599453, 0.6931471805599453)

In [31]:
(1-1)*np.log(1-myprob(100))

nan

In [14]:
np.log(np.e)

1.0

In [ ]:
# about data

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices(np.array([1.0, 2.0, 3.0, 4.0, 5.0]))
iterator = dataset.make_one_shot_iterator()

one_element = iterator.get_next()

with tf.Session(config=config) as sess:
    for i in range(5):
        print(sess.run(one_element))

In [ ]:
dataset

In [ ]:
iterator

In [ ]:
one_element

In [ ]:
rootpath = tf.constant("/home/llhan/workspace/learn_dl/bs/dataset2014_bak/dataset/", dtype=tf.string)

In [ ]:
import collections

In [ ]:
np.array([0.048719, 0.123286, 0.048719,
0.123286, 0.311981, 0.123286,
0.048719, 0.123286, 0.048719]).reshape(3,3)

In [ ]:
a = tf.placeholder(dtype=tf.float32, shape=[5,5,1])
b = tf.placeholder(dtype=tf.float32, shape=[3,3,1,1])
tf.nn.conv2d(a,b, strides=[1,1,1,1], padding="SAME")

In [ ]:
gaussian_kernel = tf.constant(np.array([0.048719, 0.123286, 0.048719,
0.123286, 0.311981, 0.123286,
0.048719, 0.123286, 0.048719]).reshape(3,3,1,1), dtype=tf.float32)

In [ ]:
gaussian_kernel.shape

In [ ]:
a = gaussian_kernel[0::2,0::2,:,:]

In [ ]:
zip((1,2,3),(4,5,6),(7,8,9))

In [ ]:
def _parse(filenames):
    
    print filenames.shape, filenames.dtype
    relpaths = tf.decode_csv(filenames, [[""],[""],[""],[""]])
#     print relpaths.shape, relpaths.dtype
    print type(relpaths), len(relpaths), relpaths[0].shape, relpaths[0].dtype
#     return relpaths
    
    fullpaths = [tf.reduce_join([rootpath, i]) for i in relpaths]
    img_strings = [tf.read_file(i) for i in fullpaths]
    img_decode = [tf.image.decode_image(i) for i in img_strings]  #prev, curr, bkg is [natch, height, width, 3], gt is [batch, hegith, width, 1]
#     return img_decode
    gt = img_decode[-1]
    img_decode = img_decode[:-1]
    
    scale0 = [tf.cast(tf.image.rgb_to_grayscale(i), tf.float32) for i in img_decode]  # rgb2gray， channels is 1, shape=[?,?,1]
    scale1 = [tf.nn.conv2d(i, gaussian_kernel, strides=[1,1,1,1], padding="SAME") for i in scale0] # if using dataset.batch, shape=[batch_size,?,?,1]
#     scale1 = [i[0::2, 0::2, :, :] for i in scale1]
#     scale2 = [tf.nn.conv2d(i, gaussian_kernel, strides=[1,1,1,1], padding="SAME") for i in scale1]
#     scale2 = [i[0::2, 0::2, :, :] for i in scale2]
#     scale0 = tf.concat(scale0, axis=2)
#     scale1 = tf.concat(scale1, axis=2)
#     scale2 = tf.concat(scale2, axis=2)
    
#     gt = gt[:,:,0]
#     gt = tf.
    
    return scale1 + [gt]

In [ ]:
with tf.Session(config=config) as s:
    print(s.run(tf.stack([rootpath]*5, axis=0)))

In [ ]:
tf.constant(value=["c,b,a", "c,b,a"], dtype=tf.string)

In [ ]:
tf.constant(value=[["a,b,c"],["c,b,a"]], dtype=tf.string)

In [ ]:
with tf.Session(config=config) as sess:
    a = sess.run(tf.decode_csv(["a,b,c", "c,b,a"], [[""],[""],[""]]))
    b = sess.run(tf.decode_csv([["a,b,c"],["c,b,a"]], [[""],[""],[""]]))

In [ ]:
a[2], b[2]

In [ ]:
with tf.Session(config=config) as sess:
    a = sess.run(tf.reduce_join([["a","b"],["c","d"]],axis=0))

In [ ]:
a = tf.placeholder(dtype=tf.float32, shape=[3,None, 3])

In [ ]:
tf.unstack(a)

In [ ]:
a = tf.constant(["/home/llhan/workspace/learn_dl/bs/dataset2014_bak/dataset/baseline/highway/input/in000570.jpg", 
                 "/home/llhan/workspace/learn_dl/bs/dataset2014_bak/dataset/baseline/highway/groundtruth/gt000570.png"], dtype=tf.string)
tf.read_file(a[0])

In [ ]:
gaussian_kernel

In [ ]:
a = tf.placeholder(dtype=tf.float32, shape=[2,3])
b = tf.reshape(a, [-1, 6])

In [ ]:
b.shape

In [ ]:
def _parse3(filenames):
    relpaths = tf.decode_csv(filenames, [[""],[""],[""],[""]])  # a list of 1-D tensor, each element is a batch of prevs, or currs, or bkgs, or gts.
    print relpaths[0].dtype, relpaths[0].shape
#     return relpaths
    
    prev_path = tf.reduce_join([rootpath, relpaths[0]])
    print prev_path.shape
    prev_img = tf.reshape(tf.cast(tf.image.rgb_to_grayscale(tf.image.decode_image(tf.read_file(prev_path))), dtype=tf.float32), shape=[-1, 240, 320, 1])
    print prev_img.shape
    
    prev_img_downscale1 = tf.nn.conv2d(prev_img,  gaussian_kernel, strides=[1,1,1,1], padding="SAME")[:,0::2,0::2,:]
    print prev_img_downscale1
    
    prev_img_downscale2 = tf.nn.conv2d(prev_img_downscale1, gaussian_kernel, strides=[1,1,1,1], padding="SAME")[:,0::2,0::2,:]
    print prev_img_downscale2
    
    prev_img = tf.reshape(prev_img, shape=[240, 320, 1])
    prev_img_downscale1 = tf.reshape(prev_img_downscale1, shape=[120, 160, 1])
    prev_img_downscale2 = tf.reshape(prev_img_downscale2, shape=[60, 80, 1])
    
    
    return prev_img_downscale2

    
    

In [ ]:
batch_size = 4
# rootpath1 = tf.stack([rootpath]*batch_size, axis=0)

csv = tf.data.TextLineDataset("./bs/mybs/train.csv")
csv = csv.map(_parse3)
csv = csv.shuffle(buffer_size=1000)   # 先 shuffle 后 map， 最后 batch。 还是先 map, 然后shuffle，最后batch
csv = csv.batch(batch_size)

iterator_csv = csv.make_one_shot_iterator()
one_element_csv = iterator_csv.get_next()

In [ ]:
one_element_csv

In [ ]:
with tf.Session(config=config) as sess:
    a = sess.run(one_element_csv)

In [ ]:
a.shape

In [ ]:
a[1]

In [ ]:
######################################################################################################

In [ ]:
def _parse2(filenames):
    def get_pyramid(relpaths):
        fullpaths = tf.reduce_join([rootpath1, relpaths], axis=0)
        imgs = [tf.cast(tf.image.rgb_to_grayscale(tf.image.decode_image(tf.read_file(fullpaths[i]))), dtype=tf.float32)  \
                for i in range(fullpaths.shape.as_list()[0])]
        imgs_downscale1 = tf.nn.conv2d(imgs, gaussian_kernel, strides=[1,1,1,1], padding="SAME")[:,0::2,0::2,:]
        imgs_downscale2 = tf.nn.conv2d(imgs_downscale1, gaussian_kernel, strides=[1,1,1,1], padding="SAME")[:,0::2,0::2,:]
        return imgs, imgs_downscale1, imgs_downscale2
    
    def get_gt_and_mask(relpaths):
        gt_paths = tf.reduce_join([rootpath1, relpaths], axis=0)    #1. 读取并解码，转换为2-Dtensor，转换为二值图, 转换为float类型

        raw_gt_imgs = [tf.image.decode_image(tf.read_file(gt_paths[i])) for i in range(gt_paths.shape.as_list()[0])]
        gt_imgs = [tf.cast(tf.greater_equal(i, 170), dtype=tf.float32) for i in raw_gt_imgs]
        gt_imgs = tf.stack(gt_imgs, axis=0)
        gt_imgs = tf.squeeze(gt_imgs)

        mask_imgs = [tf.cast(tf.not_equal(i, 85), dtype=tf.float32) for i in raw_gt_imgs]
        mask_imgs = tf.stack(mask_imgs, axis=0)
        mask_imgs = tf.squeeze(mask_imgs)
        return gt_imgs, mask_imgs

    print type(filenames), filenames.shape                      # 1-D tensor, 
    relpaths = tf.decode_csv(filenames, [[""],[""],[""],[""]])  # a list of 1-D tensor, each element is a batch of prevs, or currs, or bkgs, or gts.
    print type(relpaths), len(relpaths), relpaths[0].shape, relpaths[0].dtype
#     return relpaths
    
    prev = get_pyramid(relpaths[0])
    curr = get_pyramid(relpaths[1])
    bkg  = get_pyramid(relpaths[2])
    gt, mask = get_gt_and_mask(relpaths[3])
    
    scale0 = tf.concat([prev[0], curr[0], bkg[0]], axis=3)
    scale1 = tf.concat([prev[1], curr[1], bkg[1]], axis=3)
    scale2 = tf.concat([prev[2], curr[2], bkg[2]], axis=3)
    
    return scale0, scale1, scale2, gt, mask

In [ ]:
def _parse1(filenames):
    
    print type(filenames), filenames.shape                      # 1-D tensor, 
    relpaths = tf.decode_csv(filenames, [[""],[""],[""],[""]])  # a list of 1-D tensor, each element is a batch of prevs, or currs, or bkgs, or gts.
    print type(relpaths), len(relpaths), relpaths[0].shape, relpaths[0].dtype
#     return relpaths
    
    prev_paths = tf.reduce_join([rootpath1, relpaths[0]],axis=0)  #reduce_join, axis=0表示列方向，1-D tensor is viewed as 2-D tensor with row equaling 1
    print type(prev_paths), prev_paths.shape, prev_paths.dtype
#     return prev_paths

    prev_imgs = [tf.cast(tf.image.rgb_to_grayscale(tf.image.decode_image(tf.read_file(prev_paths[i]))), dtype=tf.float32)  \
                 for i in range(prev_paths.shape.as_list()[0])]  #read_file, decode_img, rgb_to_grayscale, cast. grayscale is 3-D tensor with channels equaling 1
    print type(prev_imgs), prev_imgs[0].shape, prev_imgs[0].dtype
#     return prev_imgs

    prev_imgs = tf.stack(prev_imgs, axis=0)
    print prev_imgs.shape, prev_imgs.dtype
#     return prev_imgs

    prev_imgs_downscale1 = tf.nn.conv2d(prev_imgs, gaussian_kernel, strides=[1,1,1,1], padding="SAME")
    print prev_imgs_downscale1.dtype, prev_imgs_downscale1.shape
    prev_imgs_downscale1 = prev_imgs_downscale1[:,0::2,0::2,:]
#     return prev_imgs_downscale1
    
    prev_imgs_downscale2 = tf.nn.conv2d(prev_imgs_downscale1, gaussian_kernel, strides=[1,1,1,1], padding="SAME")
    print prev_imgs_downscale2.dtype, prev_imgs_downscale2.shape
#     return prev_imgs_downscale2
    prev_imgs_downscale2 = prev_imgs_downscale2[:,0::2,0::2,:]
#     return prev_imgs_downscale2

    gt_paths = tf.reduce_join([rootpath1, relpaths[3]], axis=0)    #1. 读取并解码，转换为2-Dtensor，转换为二值图, 转换为float类型
#     return gt_paths
    raw_gt_imgs = [tf.image.decode_image(tf.read_file(gt_paths[i])) for i in range(gt_paths.shape.as_list()[0])]
    gt_imgs = [tf.cast((i >= 170), dtype=tf.float32) for i in raw_gt_imgs]
    gt_imgs = tf.stack(gt_imgs, axis=0)
#     return gt_imgs
    
    print raw_gt_imgs[0].shape, raw_gt_imgs[0].dtype
    mask_imgs = [tf.cast(tf.not_equal(i, 85), dtype=tf.float32) for i in raw_gt_imgs]
    mask_imgs = tf.stack(mask_imgs, axis=0)
    return mask_imgs

In [ ]:
batch_size = 10
rootpath1 = tf.stack([rootpath]*batch_size, axis=0)

csv = tf.data.TextLineDataset("./bs/mybs/train.csv")
csv = csv.shuffle(buffer_size=1000)   # 先 shuffle 后 batch， 最后 parse
csv = csv.batch(batch_size)

csv = csv.map(_parse2)
iterator_csv = csv.make_one_shot_iterator()
one_element_csv = iterator_csv.get_next()

In [ ]:
one_element_csv

In [ ]:
with tf.Session(config=config) as sess:
#     a=sess.run(one_element_csv)
#     a,b,c,d = sess.run(one_element_csv)
    a,b,c,d,e = sess.run(one_element_csv)

In [ ]:
a.shape, b.shape, c.shape, d.shape, e.shape

In [ ]:
a.dtype, b.dtype, c.dtype, d.dtype, e.dtype

In [ ]:
cv2.destroyAllWindows()

In [ ]:
i = 2
cv2.imshow("a",a[i, :,:, 0].astype(np.uint8))
cv2.imshow("b",a[i, :,:, 1].astype(np.uint8))
# cv2.imshow("c",a[1, :,:, 2].astype(np.uint8))
cv2.imshow("gt", d[i, :,:].astype(np.uint8)*255)
cv2.imshow("mask", e[i, :,:].astype(np.uint8)*255)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
skimage.io.imshow(a[0][:,:,0])

In [ ]:
a[0].shape, a[0].dtype, a[1].shape, a[1].dtype, a[2].shape, a[2].dtype

In [ ]:
cv2.imshow("a", a[0][0,:,:,0].astype(np.uint8))
cv2.imshow("b", a[1][0,:,:,0].astype(np.uint8))
cv2.imshow("c", a[2][0,:,:,0].astype(np.uint8))
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
a[0].shape, a[0].dtype

In [ ]:
a[0].shape, a[0].dtype

In [ ]:
cv2.imshow("a",a[0,:,:,0].astype(np.uint8))
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
skimage.io.imshow(a[0][:,:,0].astype(np.uint8))

In [ ]:
a.shape

In [ ]:
b.shape

In [ ]:
c.shape

In [ ]:
d.shape

In [ ]:
skimage.io.imshow(a[0])

In [ ]:
(a[1] == b[0]).sum()

In [ ]:
skimage.io.imshow(a[1])

In [ ]:
skimage.io.imshow(b[0])

In [ ]:
skimage.io.imshow(b[1])

In [ ]:
cv2.imshow("d", d[:,:,0])
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
a.shape

In [ ]:
cv2.imshow("a",a[:,:,::-1])
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
skimage.io.imshow(a[:,:,0])

In [ ]:
with tf.Session(config=config) as sess:
    for i in range(21):
        a,b,c,d = sess.run(one_element_csv)
#         print a.shape, b.shape, c.shape, d.shape
#         print a.dtype, b.dtype, c.dtype, d.dtype
        aes.append(a)
        bes.append(b)
        ces.append(c)
        des.append(d)
        
#         skimage.io.imshow(a)
#         skimage.io.imshow(b)
#         skimage.io.imshow(c)
#         skimage.io.imshow(np.concatenate([d,d,d], axis=2))

In [ ]:
aes[0].shape

In [ ]:
bes

In [ ]:
skimage.io.imshow(des[9][:,:,0])

In [ ]:
with tf.Session(config=config) as s:
    c = s.run(b)

In [ ]:
c

In [ ]:
csv = open("")

In [ ]:
a = tf.read_file("./juri1.jpg")
b = tf.image.decode_image(a)

In [ ]:
with tf.Session(config=config) as s:
    c = s.run(b)

In [ ]:
skimage.io.imshow(c)

In [ ]:
cv2.imshow("c", c[:,:,::-1])
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
# about default_graph
print tf.get_default_graph()

In [ ]:
tf.reset_default_graph()

In [ ]:
print tf.get_default_graph()

In [ ]:
##################################################

In [5]:
#about total_variation
x = tf.constant([[[[1],[2],[3]],[[6],[5],[4]],[[8],[7],[9]]], [[[1],[2],[3]],[[6],[5],[4]],[[8],[7],[8]]]])
x_var = tf.image.total_variation(x)

In [7]:
x.shape

TensorShape([Dimension(2), Dimension(3), Dimension(3), Dimension(1)])

In [8]:
x_var.shape

TensorShape([Dimension(2)])

In [10]:
s.run([x,x_var])

[array([[[[1],
          [2],
          [3]],
 
         [[6],
          [5],
          [4]],
 
         [[8],
          [7],
          [9]]],
 
 
        [[[1],
          [2],
          [3]],
 
         [[6],
          [5],
          [4]],
 
         [[8],
          [7],
          [8]]]], dtype=int32), array([25, 23], dtype=int32)]

In [ ]:
npx = np.array([[[1],[2],[3]],[[6],[5],[4]],[[8],[7],[9]]])

In [ ]:
x.shape

In [ ]:
npx[1:,:,:]

In [ ]:
gx = npx[1:,:,:] - npx[:-1,:,:]
gy = npx[:,1:,:] - npx[:,:-1,:]

In [ ]:
np.abs(gx)

In [ ]:
np.abs(gy)

In [ ]:
np.abs(gx).sum() + np.abs(gy).sum()

In [ ]:
x = tf.placeholder(dtype=tf.int32, shape=[None, 3, 3])
x_var = tf.image.total_variation(x)

In [ ]:
x.shape

In [ ]:
x_var.shape

In [ ]:
a = tf.shape(x)

In [ ]:
a

In [ ]:
a.shape

In [ ]:
# about resize_images
x = tf.placeholder(dtype=tf.float32, shape=[2,3,4,1])
resize_x = tf.image.resize_images(x, size=[6,8])

In [ ]:
tf.Tensor([3,4], dtype=tf.float32, value_index=0)

In [ ]:
x.shape[1:]

In [ ]:
a = x.shape

In [ ]:
s.run(a)

In [ ]:
s.run(tf.shape(a))

In [ ]:
# about shape
t = tf.constant([[[1, 1, 1], [2, 2, 2]], [[3, 3, 3], [4, 4, 4]]])
t_shp = tf.shape(t)

In [ ]:
s.run(t_shp)

In [ ]:
# about conv2d_transpose

In [ ]:
with tf.Graph().as_default():
    with tf.Session(config=config) as s:
        scale0 = tf.placeholder(dtype=tf.float32, shape=[None, 60, 80, 512])
        scale1 = tf.placeholder(dtype=tf.float32, shape=[None, 30, 40, 512])
        w = tf.get_variable("w", shape=[3,3,512,512], dtype=tf.float32, initializer=tf.random_normal_initializer())
        init_op = tf.global_variables_initializer()

In [ ]:
deconv1 = tf.nn.conv2d_transpose(scale1, w, output_shape=tf.shape(scale0), strides=[1,2,2,1], padding="SAME")

In [ ]:
s.run(init_op)

In [ ]:
a = s.run(deconv1, feed_dict={scale1:np.random.rand(2,30,40,512), scale0:np.random.rand(2,60,80,512)})

In [ ]:
a.shape

In [ ]:
deconv.shape

In [ ]:
deconv_reshp = tf.reshape(deconv, shape=[-1]+y.shape.as_list()[1:])

In [ ]:
deconv_reshp.shape

In [ ]:
# batch_size = tf.shape(y)[0]
deconv1_shape = tf.stack([a[0], a[1], a[2], a[3]])
deconv1 = tf.nn.conv2d_transpose(x, w, output_shape=deconv1_shape, strides=[1,2,2,1], padding="SAME")

In [ ]:
deconv1.shape

In [ ]:
int(x.shape[1])

In [ ]:
from tensorflow.contrib import slim

inputs = tf.random_normal(shape=[3, 97, 97, 10])

conv1 = slim.conv2d(inputs, num_outputs=20, kernel_size=3, stride=4)

de_weight = tf.get_variable('de_weight', shape=[3, 3, 10, 20])

deconv1 = tf.nn.conv2d_transpose(conv1, filter=de_weight, output_shape=tf.shape(inputs),
                                 strides=[1, 3, 3, 1], padding='SAME')

In [ ]:
conv1.shape

In [ ]:
deconv1.shape

In [ ]:
npx = np.random.randint(10, size=(2,8,8,3)).astype(np.float32)

In [ ]:
init_op = tf.global_variables_initializer()
s.run(init_op)

In [ ]:
tf_deconv = s.run(deconv, feed_dict={x:npx})

In [ ]:
tf_deconv.shape

In [ ]:
# about reshape
x = tf.placeholder(dtype=tf.int32, shape=[None, 3, 4, 4, 3])
x_reshp = tf.reshape(x, shape=[-1, 4, 4, 3])
x_reshp_reshp = tf.reshape(x_reshp, shape=[-1, 3, 4, 4, 3])

In [ ]:
a = np.array(range(2*3*4*4*3)).reshape((2,3,4,4,3))

In [ ]:
tfa, tfa_reshp, tfa_reshp_reshp = s.run([x,x_reshp,x_reshp_reshp], feed_dict={x: a})

In [ ]:
tfa.shape

In [ ]:
tfa_reshp.shape

In [ ]:
(tfa_reshp_reshp == tfa).sum()

In [ ]:
x.shape

In [ ]:
print x.shape

In [ ]:
# about dynamic rnn
x = tf.placeholder(dtype=tf.float32, shape=[None, 4, 10])
x_splt = tf.unstack(x, axis=1)
rnn_cell = tf.nn.rnn_cell.BasicRNNCell(3)
# init_state = tf.zeros_like(x, dtype=tf.float32)
# init_state = tf.zeros(dtype=tf.float32, shape=[None, rnn_cell.state_size])

In [ ]:
a = x.shape[0].value

In [ ]:
a

In [ ]:
x.shape[0].value

In [ ]:
rnn_cell.zero_state(3, dtype=tf.float32)

In [ ]:
x_splt

In [ ]:
tf.reset_default_graph()

In [ ]:
tf.nn.dynamic_rnn(rnn_cell, x, dtype=tf.float32)

In [ ]:
rnn_cell.state_size

In [ ]:
# about pair wise l2 dist
npa = np.array([[1,2],[3,4]])
npb = np.array([[5,6],[7,8], [9, 10]])

In [ ]:
M = a.dot(b.T)
te = np.square(a).sum(axis=1)
tr = np.square(b).sum(axis=1)
print M.shape, te.shape, tr.shape, np.matrix(te).shape

In [ ]:
dist_mat = -2 * M + tr + np.matrix(te).T

In [ ]:
dist_mat

In [ ]:
a = tf.placeholder(dtype=tf.float32, shape=[None, 2])
b = tf.placeholder(dtype=tf.float32, shape=[None, 2])
ra = tf.reshape(tf.reduce_sum(tf.square(a), axis=1), shape=[-1,1])
rb = tf.reshape(tf.reduce_sum(tf.square(b), axis=1), shape=[-1,1])
dist_mat = ra + tf.transpose(rb) - 2 * tf.matmul(a, tf.transpose(b))

In [ ]:
dist_mat.shape

In [ ]:
ra.shape

In [ ]:
rb.shape

In [ ]:
s.run(dist_mat, feed_dict={a:npa, b:npb})

In [ ]:
# about decay
global_step = tf.Variable(0, trainable=False)
initial_learning_rate = 0.1 #初始学习率
learning_rate = tf.train.exponential_decay(initial_learning_rate,
                                           global_step=global_step,
                                           decay_steps=10,decay_rate=0.9)
# opt = tf.train.GradientDescentOptimizer(learning_rate)

add_global = global_step.assign_add(1)
s.run(tf.global_variables_initializer())
print(s.run(learning_rate))
for i in range(10):
    add_g, rate = s.run([add_global, learning_rate])
    print add_g, rate

In [ ]:
0.1 * pow(0.9, 9/10.)

In [ ]:
# about concat
x1 = tf.placeholder(dtype=tf.float32, shape=[None, 28, 28, 4])
x2 = tf.placeholder(dtype=tf.float32, shape=[None, 28, 28, 5])
c = tf.concat([x1, x2], axis=3)

In [ ]:
tf.shape(x1)

In [ ]:
x1.shape.as_list()

In [ ]:
c.shape

In [ ]:
# about conv gru

class ConvGRUCell(tf.nn.rnn_cell.RNNCell):
    """
    Note that:
    1. number of channels of update gate, reset gate, hidden state and new state must be equal
    2. height and width of hidden state, input and new state must be equal
    """
    def __init__(self, ix_channels, ih_channels, kernel_shape):
        self._ix_channels = ix_channels
        self._ih_channels = ih_channels
        self._kernel_shape = kernel_shape
        
    def __call__(self, inputs, state, scope=None):
        scope = scope or type(self).__name__
        with tf.variable_scope(scope):
            in_channels = self._ix_channels + self._ih_channels
            shape_length = len(inputs.shape.as_list())
            print "shape_length", shape_length

            W_ur = tf.get_variable("W_ur", shape=self._kernel_shape+[in_channels, 2*self._ih_channels], dtype=tf.float32, initializer=tf.contrib.layers.xavier_initializer(seed=0))
            W_h  = tf.get_variable("W_h",  shape=self._kernel_shape+[in_channels,   self._ih_channels], dtype=tf.float32, initializer=tf.contrib.layers.xavier_initializer(seed=0))
            
            b_ur = tf.get_variable("b_ur", shape=[2*self._ih_channels], dtype=tf.float32, initializer=tf.contrib.layers.xavier_initializer(seed=0))
            b_h  = tf.get_variable("b_h",  shape=[  self._ih_channels], dtype=tf.float32, initializer=tf.contrib.layers.xavier_initializer(seed=0))
            
            ur = tf.nn.sigmoid(tf.nn.conv2d(tf.concat(values=[inputs, state], axis=shape_length-1),
                                            W_ur, strides=[1,1,1,1], padding="SAME") + b_ur)
            print "ur.shape", ur.shape
            u, r = tf.split(ur, num_or_size_splits=2, axis=shape_length-1)
            print "u and r's shape", u.shape, r.shape
            
            r_state = r * state
            print "r_state.shape", r_state.shape
            c = tf.tanh(tf.nn.conv2d(tf.concat(values=[inputs, r_state], axis=shape_length-1),
                                     W_h, strides=[1,1,1,1], padding="SAME") + b_h)
            print "c.shape", c.shape
            new_state = u * state + (1-u) * c
            print "new_state", new_state.shape
        self.wur, self.wh, self.bur, self.bh, self.u, self.r, self.r_state, self.c = W_ur, W_h, b_ur, b_h, u, r, r_state, c
        return new_state, new_state

In [ ]:
tf.reset_default_graph()

In [ ]:
x = tf.placeholder(dtype=tf.float32, shape=[None, 5, 5, 3])
init_state = tf.placeholder(dtype=tf.float32, shape=[None, 5, 5, 4])
cell = ConvGRU(3, 4, [3,3])
tfo, tfns = cell(x, init_state)

In [ ]:
cell.wur.graph

In [ ]:
assert cell.wur.graph is tf.get_default_graph()

In [ ]:
s.graph

In [ ]:
tf.get_default_graph()

In [ ]:
tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope="ConvGRU")

In [ ]:
npx = np.random.randint(10, size=(2,5,5,3))
nph = np.random.randint(10, size=(2,5,5,4))
feed_dict={x: npx, init_state: nph}

In [ ]:
s = tf.Session(graph=cell.wur.graph)
init_op = tf.global_variables_initializer()
s.run(init_op)

In [ ]:
npo, npns, wur, wh, bur, bh, u, r, r_state, c = s.run([tfo, tfns, cell.wur, cell.wh, cell.bur, cell.bh, cell.u, cell.r, cell.r_state, cell.c], feed_dict=feed_dict)

In [ ]:
import np_conv2d

In [ ]:
np.concatenate([npx, nph], axis=3).shape

In [ ]:
ur = 1.0/ (1+np.exp(-1*(np_conv2d.conv2d(np.concatenate([npx, nph], axis=3), wur) + bur)))

In [ ]:
ur.shape

In [ ]:
npu = ur[:,:,:,:4]
npr = ur[:,:,:,4:]

In [ ]:
np.isclose(npu, u).sum()

In [ ]:
np.isclose(npr, r).sum()

In [ ]:
np.isclose(r*nph, r_state).sum()

In [ ]:
npc = np.tanh(np_conv2d.conv2d(np.concatenate([npx, r_state],axis=3), wh) + bh)

In [ ]:
np.isclose(npc, c).sum()

In [ ]:
npc

In [ ]:
c

In [ ]:
# about conv rnn
class Conv_RNNCell(tf.nn.rnn_cell.RNNCell):
    def __init__(self, ix_channels, ih_channels, o_channels, wx_size, wh_size):
        self.ix_channels = ix_channels
        self.ih_channels = ih_channels
        self.o_channels = o_channels
        
        assert wx_size == wh_size
        self.wx_size = wx_size
        self.wh_size = wh_size
    
    def __call__(self, inputs, state, scope=None):
        scope = scope or type(self).__name__
        with tf.variable_scope(scope):
            W_x = tf.get_variable("W_x", shape=[self.wx_size, self.wx_size, self.ix_channels, self.o_channels], 
                                  dtype=tf.float32, initializer=tf.truncated_normal_initializer(mean=0.0, stddev=1e-3))
            W_h = tf.get_variable("W_h", shape=[self.wh_size, self.wh_size, self.ih_channels, self.o_channels], 
                                  dtype=tf.float32, initializer=tf.truncated_normal_initializer(mean=0.0, stddev=1e-3))
            b = tf.get_variable("b", shape=[self.o_channels], 
                                  dtype=tf.float32, initializer=tf.contrib.layers.xavier_initializer())
            
            w_conv_x = tf.nn.conv2d(inputs, W_x, strides=[1,1,1,1], padding="SAME")
            w_conv_h = tf.nn.conv2d(state,  W_h, strides=[1,1,1,1], padding="SAME")
            new_state = tf.nn.tanh(w_conv_x + w_conv_h + b)
            
            self.wx = W_x
            self.wh = W_h
            self.b  = b
        output = new_state
        return output, new_state        

In [ ]:
tf.reset_default_graph()

In [ ]:
x = tf.placeholder(dtype=tf.float32, shape=[None, 5, 5, 1])
init_state = tf.placeholder(dtype=tf.float32, shape=[None, 5, 5, 2])
conv_rnn_cell = Conv_RNNCell(1, 2, 3, 5, 5)
output, new_state = conv_rnn_cell(x, init_state)

In [ ]:
a = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope="Conv_RNNCell")[0]

In [ ]:
assert x.graph is init_state.graph
assert x.graph is output.graph
assert x.graph is tf.get_default_graph()

In [ ]:
npx = np.random.randint(10, size=(2,5,5,1))
nph = np.random.randint(10, size=(2,5,5,2))
feed_dict={x: npx, init_state: nph}

In [ ]:
init_op1 = tf.global_variables_initializer()

In [ ]:
s.run(init_op1)

In [ ]:
npo, npns, wx, wh, b = s.run([output, new_state, conv_rnn_cell.wx, conv_rnn_cell.wh, conv_rnn_cell.b], feed_dict=feed_dict)

In [ ]:
output.shape

In [ ]:
import np_conv2d
reload(np_conv2d)

In [ ]:
d = np_conv2d.conv2d(npx, wx)
e = np_conv2d.conv2d(nph, wh)
c = np.tanh(d + e + b)

In [ ]:
np.isclose(c, npo).sum()

In [ ]:
c

In [ ]:
npo

In [ ]:
# about rnn_cell
class RNNCell(tf.nn.rnn_cell.RNNCell):
    def __init__(self, input_size, state_size):
        self.input_size = input_size
        self._state_size = state_size

    @property
    def state_size(self):
        return self._state_size

    @property
    def output_size(self):
        return self._state_size
        
    def __call__(self, inputs, state, scope=None):
        scope = scope or type(self).__name__
        with tf.variable_scope(scope):
            W_x = tf.get_variable('W_x', shape=[self.input_size, self._state_size], dtype=tf.float32, 
                                initializer=tf.truncated_normal_initializer(mean=0.0, stddev=1e-3))
            W_h = tf.get_variable('W_h', shape=[self._state_size, self._state_size], dtype=tf.float32,
                                 initializer=tf.truncated_normal_initializer(mean=0.0, stddev=1e-3))
            b = tf.get_variable('b', shape=[self._state_size], dtype=tf.float32, 
                                  initializer=tf.contrib.layers.xavier_initializer())
            new_state = tf.nn.tanh(tf.matmul(inputs, W_x) + tf.matmul(state, W_h) + b)
            print W_x.name, W_h.name
        output = new_state
        self.wx, self.wh, self.b = (W_x, W_h, b)
        return output, new_state

In [ ]:
rnn = RNNCell(3,2)
x = tf.placeholder(dtype=tf.float32, shape=[None, 3])
state = tf.placeholder(dtype=tf.float32, shape=[None, 2])
output, new_state = rnn(x, state)

In [ ]:
np.dot()

In [ ]:
npx = np.array([[1,2,3]])
nph = np.array([[4,5]])

In [ ]:
init_op = tf.global_variables_initializer()
s.run(init_op)
npo, npns, wx, wh, b = s.run([output, new_state, rnn.wx, rnn.wh, rnn.b], feed_dict={x:npx, state:nph})

In [ ]:
print npo, npns

In [ ]:
np.tanh(npx.dot(wx) + nph.dot(wh) + b)

In [ ]:
tf.reset_default_graph()

In [ ]:
cell = tf.nn.rnn_cell.BasicRNNCell(2)
x1 = tf.placeholder(dtype=tf.float32, shape=[None, 3])
state1 = tf.placeholder(dtype=tf.float32, shape=[None, 2])
output1, new_state1 = cell(x1, state1)

In [ ]:
cell.weights

In [ ]:
s.run([output1, new_state1, cell.weights], feed_dict={x1:npx, state1:nph})

In [ ]:
np.tanh(np.concatenate([npx, nph], axis=1).dot(np.array([[-0.60721934,  0.52676451],
         [-0.20876437, -0.06029028],
         [-0.13075608, -0.82820529],
         [ 0.18204689, -0.1674763 ],
         [-0.03953016, -0.1331923 ]], dtype=np.float32)) + np.array([0.,0.]))

In [ ]:
# about max_pool
a = tf.placeholder(dtype=tf.float32, shape=[1,3,3,1])
b = tf.nn.max_pool(a, ksize=[1,2,2,1], strides=[1,2,2,1], padding='VALID')

In [ ]:
npa = np.array([1,2,3])

In [ ]:
# about split
a = tf.placeholder(dtype=tf.float32, shape=[None, 128, 48, 3])
b,c,d,e = tf.split(a, num_or_size_splits=[32,32,32,32], axis=1)

In [ ]:
print b.shape, c.shape

In [ ]:
npa = np.array([1,2,3,4,5,6,7,8,9,0]).reshape(5, 2)
npb, npc = s.run([b,c], feed_dict={a:npa})

In [ ]:
npa

In [ ]:
npb

In [ ]:
# about conv
a = tf.placeholder(dtype=tf.float32, shape=[None,7,7,512])
w = tf.placeholder(dtype=tf.float32, shape=[7,7,512,4096])
b = tf.nn.conv2d(a, w, strides=[1, 7, 7, 1], padding='SAME')

In [ ]:
b.shape

In [ ]:
npa = np.array([1,1,1,0,0,1,1,1,0,0,1,1,0,0,1,1]).reshape(1,4,4,1)
npw = np.array([-1,1,-1,1,1,1,1,-1,-1, 1,-1,1,-1,1,1,-1,-1,-1]).reshape(3,3,1,2)

In [ ]:
print npa.shape, npw.shape
print npa[0,:,:,0]
print npw[:,:,0,0]
print npw[:,:,0,1]

In [ ]:
npb = s.run(b, feed_dict={a:npa, w:npw})

In [ ]:
npb

In [ ]:
npb[0,:,:,1]

In [ ]:
# about pow
a = tf.placeholder(dtype=tf.float32, shape=[2])

In [ ]:
s.run(tf.pow(a,2), feed_dict={a:np.array([1,2])})

In [ ]:
# about norm
# norm(a, 2), 表示的是l2-norm，就是所有元素的平方和，再开方
a = tf.placeholder(dtype=tf.float32, shape=[1,2])

In [ ]:
s.run(tf.norm(a,2), feed_dict={a:np.array([[3,4]])})

In [ ]:
# about equal
a = tf.placeholder(dtype=tf.int32, shape=[None])
b = tf.placeholder(dtype=tf.int32, shape=[a.shape[0]])
c = tf.placeholder(dtype=tf.int32, shape=a.get_shape())
print a.shape, b.shape, c.shape

In [ ]:
d = 1 - a

In [ ]:
s.run(d, feed_dict={a: np.array([0,1,2,3,3])})

In [ ]:
e = tf.subtract(1, a)

In [ ]:
# about substract
a = tf.placeholder(dtype=tf.float32, shape=[3,2])
b = 1-a
d = tf.subtract(1., a)

In [ ]:
c = np.array([[1,2],[3,4],[5,6]])
s.run([b,d],feed_dict={a : c})

In [ ]:
# about boolean_mask
w = tf.placeholder(dtype=tf.float32, shape=[None,5])
mask = tf.placeholder(dtype=tf.bool, shape=[None, 5])
w_mask = tf.boolean_mask(w, mask)

In [ ]:
print w.shape, mask.shape, w_mask.shape

In [ ]:
a = np.random.random((10, 5))
m = np.eye(10, 5)#.astype(bool)
s.run(w_mask, feed_dict={w : a, mask : m})

In [ ]:
a

In [ ]:
tf.reset_default_graph()
# about name_score variable_scope get_variable Variable
with tf.variable_scope("foo", reuse=True):
    v = tf.get_variable("v", [1])  # v.name == "foo/v:0"
    w = tf.get_variable("w", [1])  # w.name == "foo/w:0"
    u = tf.Variable(initial_value=[1], name="u")
    print tf.get_variable_scope().name, v.name, w.name, u.name
with tf.variable_scope("foo", reuse=True):
    v1 = tf.get_variable("v")  # The same as v above.
    w1 = tf.get_variable("w")
    
    print tf.get_variable_scope().name, v1.name, w1.name, u.name
print v1 is v

In [ ]:
with tf.name_scope('name_scope_x'):
    var1 = tf.get_variable(name='var1', shape=[1], dtype=tf.float32)
    var3 = tf.Variable(name='var2', initial_value=[2], dtype=tf.float32)
    var4 = tf.Variable(name='var2', initial_value=[2], dtype=tf.float32)
#     var3 = tf.get_variable(name='var1')
print var1.name, var3.name, var4.name

- variable_scope 对get_variable 和 Variable都有效
- name_scope 只对 Variable 有效
- 如果get_variable所在的scope的reuse为True，则表示重复使用变量，提供给get_variable的变量名必须是之前声明过的
- 如果get_variable所在的scope的reuse为False，则表示不重复使用变量，提供给get_variable的变量名必须是之前没有声明过的

In [ ]:
with tf.variable_scope("fuck"):
    v = tf.Variable(initial_value=[1], name='v')
    p = tf.placeholder(tf.float32)
with tf.variable_scope("fuck"):
    v = tf.Variable(initial_value=[2], name='v')

In [ ]:
with tf.variable_scope("foo") as foo_scope:
    assert foo_scope.name == "foo"
with tf.variable_scope("bar"):
    with tf.variable_scope("baz") as other_scope:
        assert other_scope.name == "bar/baz"
        with tf.variable_scope(foo_scope) as foo_scope2:
            assert foo_scope2.name == "foo"  # Not changed.

In [ ]:
# about softmax
logits = tf.constant([.1,.3,.5,.9], dtype=tf.float32)
t = tf.nn.softmax(logits)

onehot_labels = tf.constant([0,0,0,1], dtype=tf.float32)
entropy = tf.nn.softmax_cross_entropy_with_logits(labels=onehot_labels, logits=logits)

loss_entropy = tf.losses.softmax_cross_entropy(onehot_labels=onehot_labels, logits=logits)

In [ ]:
print s.run(t)
print s.run(entropy)
print s.run(loss_entropy)

In [ ]:
# about l2_loss
a = tf.constant([1,2,3,4], dtype=tf.float32)
l2_a = tf.nn.l2_loss(a)

In [ ]:
s.run(l2_a)

In [ ]:
s.run(a)

In [ ]:
# about assign
W = tf.Variable([.3], dtype=tf.float32)
W = tf.assign(W, [-1])

In [ ]:
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    print sess.run([W, W.)

In [ ]:
# about ExponentialMovingAverage
v1 = tf.Variable(0, dtype=tf.float32)
step = tf.Variable(0, trainable=False)

ema = tf.train.ExponentialMovingAverage(0.99, step)
mainain_averages_op = ema.apply([v1])  #将 v1 添加进滑动平均的列表
with tf.Session() as sess:
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    
    print sess.run([v1, ema.average(v1)])
    
    sess.run(tf.assign(v1, 5))
    print sess.run([v1, ema.average(v1)]) #ema.average 获取shadow_variable，
    sess.run(mainain_averages_op)         #这一步是更新shadow_variable 
    print sess.run([v1, ema.average(v1)])
    
    

In [ ]:
# about embedding_lookup
train_inputs = tf.placeholder(tf.int32, shape=[2,2])
embeddings = tf.Variable(tf.random_uniform([100, 10], -1.0, 1.0))
embed = tf.nn.embedding_lookup(embeddings, train_inputs)

In [ ]:
s.run(tf.global_variables_initializer())

In [ ]:
s.run(embed, feed_dict={train_inputs : [[1,2],[2,3]]})

In [ ]:
s.run(tf.global_variables_initializer())
fetch = s.run([x, wx_b], feed_dict={x : np.random.random((100, 64))})

In [ ]:
fetch